In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import copy
import os
import yaml
import pickle
from datetime import timedelta

import sys
sys.path.append('../../')

from data.processing.processing import generate_simulated_data
from data.dataloader import SimulatedDataLoader

from main.seir.fitting import single_fitting_cycle
from utils.generic.config import read_config
from utils.fitting.loss import Loss_Calculator
from utils.fitting.smooth_jump import smooth_big_jump

from viz.data import plot_data
from utils.generic.enums.columns import *
from utils.fitting.smooth_jump import perform_smoothing_experiment
from viz.utils import setup_plt, axis_formatter

In [ ]:
config_filename = 'simulate_data.yaml'
config = read_config(config_filename)

In [ ]:
seed = 0
num_trials = 5000

In [ ]:
perform_smoothing_experiment(config, 'uniform')

In [ ]:
np.random.seed(seed)
results = []
for i in range(num_trials):
            results.append(perform_smoothing_experiment(config, 'uniform'))

In [ ]:
np.array(results).mean(axis=0)

In [ ]:
df_dict = perform_smoothing_experiment(config, 'weighted-mag', mode='plotting')
df = df_dict['df']
df_spike = df_dict['df_spike']
df_spike_smooth = df_dict['df_spike_smooth']

pickle(df_dict)

In [ ]:
with open('../../../paper/smoothing/plotting/data_sim.pkl', 'wb+') as pickle_file:
    pickle.dump(df_dict, pickle_file)

In [ ]:
plt.rcParams.update({
    'text.usetex': True,
    'font.size': 20,
    'font.family': 'Palatino',
 })

In [ ]:
from data.processing.processing import get_data_from_tracker

In [ ]:
config_filename = 'default.yaml'
config1 = read_config(config_filename)

df1 = get_data_from_tracker(state='Maharashtra', district='Mumbai', use_dataframe='data_all')['data_frame']
df1 = df1[df1['date'] <= '2020-07-07']

# Perform smoothing
print('smoothing params', config1['fitting']['data']['smooth_jump_params'])
df1_smooth, _ = smooth_big_jump(df1, config1['fitting']['data']['smooth_jump_params'])

In [ ]:
mumbai_dict = {'df_mumbai' : df1, 'df_mumbai_smooth' : df1_smooth}

with open('../../../paper/smoothing/plotting/data_mumbai.pkl', 'wb+') as pickle_file:
    pickle.dump(mumbai_dict, pickle_file)

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10), nrows=2)
ax = axs.flat[0]
ax_twin = ax.twinx()
plot_ax = ax
for comp in ['active', 'recovered']:
    if comp == 'active':
        plot_ax = ax
    else:
        plot_ax = ax_twin
    compartment = Columns.from_name(comp)
    plot_ax.plot(df[compartments['date'][0].name].to_numpy(), df[compartment.name].to_numpy(),
                 '--', color=compartment.color)
    plot_ax.plot(df_spike[compartments['date'][0].name].to_numpy(), df_spike[compartment.name].to_numpy(),
                 '-o', color=compartment.color, ms=5)
    plot_ax.plot(df_spike_smooth[compartments['date'][0].name].to_numpy(), df_spike_smooth[compartment.name].to_numpy(),
                 '-', color=compartment.color)
#     plot_ax.tick_params(axis='y', labelcolor=compartment.color)
#     plot_ax.set_ylabel(compartment.label, color=compartment.color)
#     axis_formatter(plot_ax)
legend_elements = [
    Line2D([0], [0], ls='--', color='black', label='Sim Data, Unspiked'),
    Line2D([0], [0], ls='-', marker='o', ms=5, color='black', label='Sim Data, Spiked'),
    Line2D([0], [0], ls='-', color='black', label='Smoothed Data')
]
first_legend = ax.legend(handles=legend_elements, loc='lower right')
ax.add_artist(first_legend)
legend_elements = [
    Line2D([0], [0], ls='-', color='orange', label=f'Active'),
    Line2D([0], [0], ls='-', color='green', label=f'Recovered'),
]
# ax.legend(handles=legend_elements, loc=[0.018, 0.47])
axis_formatter(ax)
# ax.set_title('Comparison of smoothing algorithm with ground truth simulated data')
# fig.savefig(f'../../../paper/plots/smoothing-simulated.pdf', format='pdf', bbox_inches='tight', pad_inches=0)

ax1 = axs.flat[1]
ax1_twin = ax1.twinx()
plot_ax = ax1
for comp in ['active', 'recovered']:
    compartment = Columns.from_name(comp)
    if comp == 'active':
        plot_ax = ax1
    else:
        plot_ax = ax1_twin
    
    plot_ax.plot(df1[compartments['date'][0].name].to_numpy(), df1[compartment.name].to_numpy(),
                 '-o', color=compartment.color)
    plot_ax.plot(df1_smooth[compartments['date'][0].name].to_numpy(), df1_smooth[compartment.name].to_numpy(),
                 '-', color=compartment.color)
#     plot_ax.tick_params(axis='y', labelcolor=compartment.color)
#     plot_ax.set_ylabel(compartment.label, color=compartment.color)
#     axis_formatter(plot_ax)
    
# ax1.set_ylim(bottom=0, top=60000)
# ax1_twin.set_ylim(bottom=-10000, top=60000)
legend_elements = [
    Line2D([0], [0], ls='-', marker='o', ms=5, color='black', label='Original Data'),
    Line2D([0], [0], ls='-', color='black', label='Smoothed Data')
]
first_legend = ax1.legend(handles=legend_elements, loc='upper left')
ax1.add_artist(first_legend)
legend_elements = [
    Line2D([0], [0], ls='-', color='orange', label=f'Active'),
    Line2D([0], [0], ls='-', color='green', label=f'Recovered'),
]
# ax1.legend(handles=legend_elements, loc=[0.018, 0.55])
axis_formatter(ax1)
# ax1.set_title('Smoothing Algorithm Illustration for Mumbai, India')
fig.tight_layout()
# fig.savefig(f'../../../paper/plots/smoothing-combined-v4.pdf', format='pdf', bbox_inches='tight', pad_inches=0)